## Constants

In [1]:
import os

file_count = 50   # TODO: change to 1000
playlists_per_file = 1000

dir_data_raw = os.path.join("..", "data", "raw")
dir_data_processed = os.path.join("..", "data", "processed")

playlist_count = file_count * playlists_per_file

Merge the data extracted from the Spotify API (csv file) with that of the Spotify Million Playlist Dataset (pickle file)

In [38]:
import os
import pandas as pd

path_audio_features = os.path.join(dir_data_raw, "tracks", "track_audio_features.csv")
audio_features = pd.read_csv(path_audio_features).set_index("id")

display(audio_features)

True


,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
id,,,,,,,,,,,,,
3zyYVItCMCjFzBHTyjrxPK,79.227,8.0,1.0,-3.406,0.466,0.856,0.3180,0.257000,0.000000,0.6750,0.531,292223.0,4.0
4jSy0HTIoC9yiwZ8OVyTCW,164.207,1.0,1.0,-7.972,0.510,0.849,0.1190,0.000546,0.000748,0.6890,0.887,315067.0,4.0
4zyqBSUFNkJ20mw1FB68gt,83.947,4.0,0.0,-22.867,0.308,0.114,0.0321,0.958000,0.902000,0.0853,0.303,350906.0,4.0
63B3TtwUzOoJoe3unMteVa,93.696,1.0,0.0,-4.166,0.660,0.943,0.2770,0.129000,0.000000,0.5570,0.599,210733.0,4.0
7y9iMe8SOB6z3NoHE2OfXl,118.384,0.0,1.0,-3.539,0.675,0.751,0.0296,0.060400,0.000000,0.0893,0.612,181279.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0s9iofqbQdsmWZQJ1TZh1c,118.061,7.0,1.0,-13.766,0.550,0.351,0.0443,0.531000,0.006020,0.1500,0.237,162826.0,4.0
5Ozvg38CnCQKmPP8tgSRA6,96.520,0.0,1.0,-4.130,0.609,0.742,0.0477,0.001310,0.000002,0.0725,0.531,256227.0,4.0
2yWsZpLZNmap92YkGkzyDq,179.985,6.0,1.0,-5.899,0.373,0.584,0.0272,0.004810,0.449000,0.1630,0.403,214667.0,3.0


Create a function to compute relevant metrics about a playlist's audio features.

In [180]:
import sys

def get_playlist_metrics(df: pd.core.frame.DataFrame):
    target_columns = list(range(2,5)) # TODO: which columns do we care about?
    def get_statistic(f) -> pd.core.frame.DataFrame:
        return df.iloc[:, target_columns].apply(f).to_frame().transpose()

    assert isinstance(df, pd.core.frame.DataFrame)
    return pd.concat({"Mean" : get_statistic(pd.Series.mean),
                      "Median" : get_statistic(pd.Series.median),
                      "Standard deviation" : get_statistic(pd.Series.std),
                      "Variance" : get_statistic(pd.Series.var)}, axis=1)

Combine extracted and calculated data into a playlist frame

In [181]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlists_metrics = pd.DataFrame()

total_processed = 0 #TODO: DELETE ME WHEN FINISHED TESTING!
total_tracks = 0 #TODO: DELETE ME WHEN FINISHED TESTING!

for i, playlist in enumerate(playlist_data[3:100]):
    track_counter = 0 # TODO: DELETE ME WHEN FINISHED TESTING!
    playlist_features = pd.DataFrame(columns = audio_features.columns.tolist())

    for track_id in playlist["track_ids"]:
        total_tracks += 1

        if track_id not in audio_features.index: # TODO: handle cases when track id has no audio features
            continue

        track_features = pd.DataFrame(audio_features.loc[track_id]).T
        track_counter += 1
        total_processed += 1
        
        playlist_features = pd.concat([playlist_features if not playlist_features.empty else None, 
                                       track_features], axis=0)
        
    print("Playlist features")
    display(playlist_features)

    # print("{:3} - Processed tracks:{:3}/{:3}".format(i, track_counter, len(playlist["track_ids"])))    
    
    playlist_info = pd.DataFrame(data={k:[v] for k,v in playlist.items() if k != "track_ids"})

    print("Calling get_playlist_metrics")
    display(get_playlist_metrics(playlist_features)) ###############

    playlist_metrics = pd.concat([playlist_info, get_playlist_metrics(playlist_features)])
    playlists_metrics = pd.concat([playlists_metrics, playlist_metrics], ignore_index=True)

print("{} Playlists | Total processed tracks: {}/{}".format(i+1, total_processed, total_tracks))

Playlist features


,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
1iDPB2WNsGEWuKycQyDGn6,137.974,1.0,1.0,-7.219,0.914,0.415,0.322,0.0270,0.0,0.1140,0.496,125217.0,4.0
3uFmweGlFudKNiX13XFwFm,124.984,4.0,1.0,-6.728,0.917,0.624,0.147,0.0579,0.0,0.0714,0.343,122996.0,4.0


Calling get_playlist_metrics


Mean                       Median                       Standard deviation  \
  mode loudness danceability   mode loudness danceability               mode   
0  1.0  -6.9735       0.9155    1.0  -6.9735       0.9155                0.0   

                         Variance                         
   loudness danceability     mode  loudness danceability  
0  0.347189     0.002121      0.0  0.120541     0.000005

Playlist features


...

Playlist features


,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
3I5vK9yUVv15WASYYlNRMA,128.745,5.0,1.0,-5.900,0.625,0.5890,0.0279,0.173,0.123000,0.1740,0.289,190015.0,4.0
3yoGVIY5OrRIBvNA9PYOS7,181.028,7.0,1.0,-14.518,0.434,0.1040,0.0342,0.823,0.000046,0.1130,0.139,182240.0,3.0
1GgYxWBBOdLFWJ4EEHIZSR,79.688,0.0,1.0,-22.067,0.476,0.2390,0.0485,0.876,0.303000,0.0921,0.125,73718.0,4.0
70su3hwMvDe2VWdQmgVIQo,55.929,9.0,1.0,-25.769,0.485,0.0241,0.0369,0.950,0.000092,0.2830,0.174,115467.0,4.0
1GgYxWBBOdLFWJ4EEHIZSR,79.688,0.0,1.0,-22.067,0.476,0.2390,0.0485,0.876,0.303000,0.0921,0.125,73718.0,4.0


Calling get_playlist_metrics


Mean                       Median                       Standard deviation  \
  mode loudness danceability   mode loudness danceability               mode   
0  1.0 -18.0642       0.4992    1.0  -22.067        0.476                0.0   

                         Variance                          
   loudness danceability     mode   loudness danceability  
0  7.936499     0.073066      0.0  62.988014     0.005339

97 Playlists | Total processed tracks: 609/7731


In [5]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlist_frame
playlist_frame = pd.DataFrame(np.arrange())

NameError: name 'playlist_frame' is not defined